## Messari standardised subgraphs

https://github.com/messari/subgraphs

### VVS Finance

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://graph.cronoslabs.com/subgraphs/name/messari/vvs-finance'

6-Month Average Daily Volume

In [3]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [4]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [5]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [6]:
df['dailyVolumeUSD'].astype(float).mean()

0.0

6-Month Average TVL

In [8]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/vvs-finance.csv")

In [9]:
lma

,Unnamed: 0,Date,Timestamp,VVS Finance,VVS Finance.1,VVS Finance.2,VVS Finance.3,VVS Finance.4,VVS Finance.5
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,cronos,Total,cronos,Total,cronos,Total
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,crypto-com-chain,crypto-com-chain,crypto-com-chain,crypto-com-chain
4,NaN,12/11/2021,1.636675e+09,289233660.20245445,289233660.20245445,289233660.20245445,289233660.20245445,822925598.77786,822925598.77786
...,...,...,...,...,...,...,...,...,...
248,NaN,14/07/2022,1.657757e+09,681537987.8634158,681537987.8634158,681537987.8634158,681537987.8634158,5862036829.11516,5862036829.11516
249,NaN,15/07/2022,1.657843e+09,685086520.6003187,685086520.6003187,685086520.6003187,685086520.6003187,5822644426.693399,5822644426.693399
250,NaN,16/07/2022,1.657930e+09,691277554.5109184,691277554.5109184,691277554.5109184,691277554.5109184,5806956767.814306,5806956767.814306
251,NaN,17/07/2022,1.658016e+09,703675140.0035027,703675140.0035027,703675140.0035027,703675140.0035027,5694408487.319259,5694408487.319259


In [10]:
lma.iloc[:,3]

0                   Dexes
1                  cronos
2                     TVL
3                     NaN
4      289233660.20245445
              ...        
248     681537987.8634158
249     685086520.6003187
250     691277554.5109184
251     703675140.0035027
252     697202060.5116348
Name: VVS Finance, Length: 253, dtype: object

In [11]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

1069543925.2167237

Largest Pool Concentration

In [12]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_19009/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [13]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0xe61db569e231b3f5530168aa2c9d50246525b6d6,Trader Joe Wrapped CRO/USD Coin,0,0,1636431821,17798
1,0xa111c17f8b8303280d3eb01bbcd61000aa7f39f9,Trader Joe Wrapped CRO/Wrapped Ether,0,0,1636431947,17821
2,0xbf62c67ea509e86f07c8c69d0286c0636c50270b,Trader Joe VVSToken/Wrapped CRO,0,0,1636431959,17823
3,0x8f09fff247b8fdb80461e5cf5e82dd1ae2ebd6d7,Trader Joe Wrapped BTC/Wrapped CRO,0,0,1636431969,17825
4,0xfd0cd0c651569d1e2e3c768ac0ffdab3c8f4844f,Trader Joe USD Coin/Wrapped Ether,0,0,1636431975,17826
...,...,...,...,...,...,...
716,0x4e7bca7171328762af4114c10dab0c1e7ca7e1d1,Trader Joe FerroToken/SOL,0,0,1657765684,3678936
717,0x3905b7cf83265d2a17af2ccd24386f02897608a3,Trader Joe WOO Network/Fantom Token,0,0,1657774500,3680488
718,0xb25b4c60b4ff9fc81d6f8f03e7a626e99b0c9114,Trader Joe VSHARE/Wrapped Ether,0,0,1657839377,3691872
719,0x4cb526465c9477773a02baeaac1c2f9de60549ce,Trader Joe VersaGames/Argo Protocol,0,0,1657912801,3704769


In [15]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.30731839237318975

In [16]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.9304788700097002

Liquidity Pools with > $1mn TVL

In [17]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

5

Monthly Active Users

In [14]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [15]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [16]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [17]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [18]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [19]:
np.mean(monthactiveusers) #can chart this easy

123099.28571428571

Implied slippage